In [2]:
import requests, json, geocoder, folium, ipywidgets, sys  # No shortage of imports here, won't use most of them for this example
from folium import plugins
from folium.plugins import FloatImage                   # Once jupyter is installed,                            # you can just type !python -m pip install ...(package)
from bs4 import BeautifulSoup as BS                     # To install any package into jupyters kernal
import pandas as pd


wiki_url = 'https://en.wikipedia.org/wiki/COVID-19_pandemic_in_New_York_(state)#Demographics'   
wiki_data = requests.get(wiki_url).content
wiki_html = BS(wiki_data, 'html.parser')          # Using Beautiful soup to parse content of page
ny_table = wiki_html.find_all('table')[4]
ny_table

<table class="wikitable plainrowheaders sortable" style="text-align:right; font-size:85%; margin:0">
<caption><div class="plainlinks hlist navbar mini" style="float:left; text-align:left"><ul><li class="nv-view"><a href="/wiki/Template:COVID-19_pandemic_data/New_York_(state)_medical_cases_by_county" title="Template:COVID-19 pandemic data/New York (state) medical cases by county"><abbr title="View this template">v</abbr></a></li><li class="nv-talk"><a href="/wiki/Template_talk:COVID-19_pandemic_data/New_York_(state)_medical_cases_by_county" title="Template talk:COVID-19 pandemic data/New York (state) medical cases by county"><abbr title="Discuss this template">t</abbr></a></li><li class="nv-edit"><a class="external text" href="https://en.wikipedia.org/w/index.php?title=Template:COVID-19_pandemic_data/New_York_(state)_medical_cases_by_county&amp;action=edit"><abbr title="Edit this template">e</abbr></a></li></ul></div><div style="font-size:114%;margin:0 4em"><span style="font-size:98%;">

In [27]:
meta_df = pd.read_html(str(ny_table))                    # Use pandas to read the html
df_json = meta_df[0].to_json(orient='records')        # format the data with pandas to_json method   
df_data = json.loads(df_json)
ny_cointies_popDF = pd.read_json(json.dumps(df_data))
ny_cointies_popDF                                   #boom we have a data frame and haven't even written a single loop


,"('County [a]', '62 / 62')","('Cases [b][c]', '356,458')","('Deaths [c]', '23,083')","('Recov. [c][d]', '78,797')","('Pop.[265]', '19,453,561')","('Cases / 100k', '1,822')","('Ref.', 'Ref.')"
0,Albany,1700,93,995,305506,423.6,[266]
1,Allegany,44,2,28,46091,75.9,None
2,Bronx,43603,3149,–,1418207,2783.5,[e]
3,Broome,451,39,326,190488,177.4,[267]
4,Cattaraugus,71,2,37,76117,69.6,[268]
5,Cayuga,72,2,51,76576,67.9,None
6,Chautauqua,58,2,37,126903,29.2,[269]
7,Chemung,134,2,10,83456,151.0,None
8,Chenango,118,3,86,47207,211.8,[270]
9,Clinton,94,4,46,80485,77.0,None


In [28]:
pd.set_option('display.max_rows', None) # view entire dataframe
nyc_c19= ny_cointies_popDF

nyc_c19.rename(columns={"('County [a]', '62 / 62')":"County","('Cases [b][c]', '356,458')": 'Cases',  
               "('Deaths [c]', '23,083')": "Deaths",      # Simply renaming the columns
               "('Recov. [c][d]', '78,797')": "Recovered",
               "('Pop.[265]', '19,453,561')":"Population",
                       "('Cases / 100k', '1,822')": "Cases/100k"},
               inplace=True, errors='raise')

nyc_c19=nyc_c19.drop([63])      # Get rid of last two rows -we dont need them
nyc_c19=nyc_c19.drop([62])
                                                                #To check data types of the dataframe: nyc_c19.dtypes
nyc_c19[["Cases","Deaths", "Recovered", "Population",           # Change data types to int or float where appropriate.
         "Cases/100k", "('Ref.', 'Ref.')"]] = nyc_c19[["Cases","Deaths", "Recovered", "Population",
                                                       "Cases/100k", "('Ref.', 'Ref.')"]].apply(pd.to_numeric, errors= 'coerce')

nyc_c19         # Our dataframe with the latest corona data provided by wiki

,County,Cases,Deaths,Recovered,Population,Cases/100k,"('Ref.', 'Ref.')"
0,Albany,1700,93,995.0,305506,423.6,NaN
1,Allegany,44,2,28.0,46091,75.9,NaN
2,Bronx,43603,3149,NaN,1418207,2783.5,NaN
3,Broome,451,39,326.0,190488,177.4,NaN
4,Cattaraugus,71,2,37.0,76117,69.6,NaN
5,Cayuga,72,2,51.0,76576,67.9,NaN
6,Chautauqua,58,2,37.0,126903,29.2,NaN
7,Chemung,134,2,10.0,83456,151.0,NaN
8,Chenango,118,3,86.0,47207,211.8,NaN
9,Clinton,94,4,46.0,80485,77.0,NaN


In [29]:
# Using a New York County GeoJson File, we now have enough data for a basic choropleth map!
# Time to use Folium


with open('Desktop/Data/GEOJSON/newyorkcounties.geojson','r') as NyGeo:
    nyGeo = json.load(NyGeo)         # Using json library to load file

# map
map_nyC19 = folium.Map(location=[43.299, -74.217], zoom_start=6)  # Define our folium map object by location, and level of zoom

# add tiles (map aesthetic) to map
folium.raster_layers.TileLayer('Stamen Toner').add_to(map_nyC19)
folium.raster_layers.TileLayer('Stamen Terrain').add_to(map_nyC19)
folium.raster_layers.TileLayer('Stamen Watercolor').add_to(map_nyC19)
folium.raster_layers.TileLayer('Open Street Map').add_to(map_nyC19)


# add feature 'id' county name to geojson
# access features
for i in nyGeo['features']:
    i['id'] = i['properties']['name']  # Depending on your file, "['name']" could have a different value, yet still point to
                                       # the name of a particular county or region. Here we just set 'id' == whatever that is


# choropleth
folium.Choropleth(geo_data=nyGeo,  # Here's where we insert the geoJson file
                name='choropleth', data=nyc_c19 ,   # Here's where we insert our pandas dataframe
                 columns=['County', 'Population'], # You need at least two columns 
                 key_on='feature.id',
                 fill_color='BuPu',
                 fill_opacity=0.5,
                 line_opacity=0.5,
                 lengend_name='Population',
                 highlight=True
                 ).add_to(map_nyC19)


folium.LayerControl().add_to(map_nyC19)    # add control for our different layers

# display map
map_nyC19






In [30]:
# Hmmm. looks like somethings going on with our geojson to cause our choropleth not to output properly.
# Oh I see! Our county names in our dataframe dont match the 'name' property in the geoJsoN file. 
# Each county name in the geoJson is followed by..." County". That's Not the case in our data frame. Lets change that.

[nyc_c19["County"].replace({county: county+" County"}, inplace=True) for county in nyc_c19['County']] #I ❤ List Comprehensions

nyc_c19

,County,Cases,Deaths,Recovered,Population,Cases/100k,"('Ref.', 'Ref.')"
0,Albany County,1700,93,995.0,305506,423.6,NaN
1,Allegany County,44,2,28.0,46091,75.9,NaN
2,Bronx County,43603,3149,NaN,1418207,2783.5,NaN
3,Broome County,451,39,326.0,190488,177.4,NaN
4,Cattaraugus County,71,2,37.0,76117,69.6,NaN
5,Cayuga County,72,2,51.0,76576,67.9,NaN
6,Chautauqua County,58,2,37.0,126903,29.2,NaN
7,Chemung County,134,2,10.0,83456,151.0,NaN
8,Chenango County,118,3,86.0,47207,211.8,NaN
9,Clinton County,94,4,46.0,80485,77.0,NaN


In [31]:
# Looks good. Let's try it Again

# map
map_nyC19 = folium.Map(location=[43.299, -74.217], zoom_start=6)  # Define our folium map object by location, and level of zoom

# add tiles (map aesthetic) to map
folium.raster_layers.TileLayer('Stamen Toner').add_to(map_nyC19)
folium.raster_layers.TileLayer('Stamen Terrain').add_to(map_nyC19)
folium.raster_layers.TileLayer('Stamen Watercolor').add_to(map_nyC19)
folium.raster_layers.TileLayer('Open Street Map').add_to(map_nyC19)


# add feature 'id' county name to geojson
# access features
for i in nyGeo['features']:
    i['id'] = i['properties']['name']  # Depending on your file, "['name']" could have a different value, yet still point to
                                       # the name of a particular county or region. Here we just set 'id' == whatever that is


# choropleth
folium.Choropleth(geo_data=nyGeo,  # Here's where we insert the geoJson file
                name='choropleth', data=nyc_c19 ,   # Here's where we insert our pandas dataframe
                 columns=['County', 'Population'], # You need at least two columns 
                 key_on='feature.id',
                 fill_color='BuPu', # Color pattern used. BuPu = blue and Purple
                 fill_opacity=0.5,
                 line_opacity=0.5,
                 lengend_name='Population',
                 highlight=True
                 ).add_to(map_nyC19)


folium.LayerControl().add_to(map_nyC19)    # add control for our different layers

# display map
map_nyC19


